In [ ]:
!pip install selenium beautifulsoup4

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv


# Start WebDriver
driver = webdriver.Safari()
wait = WebDriverWait(driver, 10)
driver.get("https://rera.odisha.gov.in/projects/project-list")
time.sleep(3)

project_data = []

try:
    for i in range(6):  # Loop over first 6 projects
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(text(),'View Details')]")))
        view_buttons = driver.find_elements(By.XPATH, "//a[contains(text(),'View Details')]")
        if i >= len(view_buttons):
            break

        button = view_buttons[i]
        wait.until(EC.element_to_be_clickable((By.XPATH, f"(//a[contains(text(),'View Details')])[{i+1}]")))
        driver.execute_script("arguments[0].click();", button)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract Project Info
        project_info = {}
        for field in ["Project Name", "RERA Regd. No."]:
            value = extract_field_value(soup, [field])
            project_info[field] = value


        # Click on Promoter Tab
        promoter_tab = driver.find_element(By.XPATH, "//a[contains(text(),'Promoter Details')]")
        promoter_tab.click()
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Fallback options for promoter info fields
        field_mappings = {
            "Company Name": ["Company Name", "Propietory Name"],
            "Registered Office Address": ["Registered Office Address", "Current Residence Address"],
            "GST No.": ["GST No."]
        }

        promoter_info = {}
        for key, label_options in field_mappings.items():
            value = extract_field_value(soup, label_options)
            promoter_info[key] = value

        # Combine and store
        project_data.append({**project_info, **promoter_info})

        # Go back to main page
        driver.back()
        time.sleep(1)

finally:
    driver.quit()

# Final Output
print("\n--- Final Extracted Data ---")
for idx, proj in enumerate(project_data, 1):
    print(f"\n- Project {idx} ---")
    for key, val in proj.items():
        print(f"{key}: {val}")

csv_filename = "/Users/shubhendumishra/Desktop/odisha_rera_projects.csv"
fieldnames = ["Project Name", "RERA Regd. No.", "Company Name", "Registered Office Address", "GST No."]

with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for row in project_data:
        writer.writerow(row)

print(f"\n✅ Data saved to '{csv_filename}' successfully.")